In [35]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

pd.options.display.max_columns = 999

# Maryland

## VEST Documentation

Election results from the Maryland State Board of Elections (https://elections.maryland.gov/elections/2020/election_data/index.html)  

Precinct shapefiles primarily sourced from the respective county governments.  

Caroline, Dorchester, Kent, Talbot, Somerset, and Worcester instead sourced from the Maryland Department of Planning.

G20PREDBID - Joseph R. Biden (Democratic Party)  
G20PRERTRU - Donald J. Trump (Republican Party)  
G20PRELJOR - Jo Jorgensen (Libertarian Party)  
G20PREGHAW - Howie Hawkins (Green Party)  
G20PREBSEG - Jerome M. Segal (Bread and Roses Party)  
G20PREOWRI - Write-in Votes  

## Load VEST File

In [36]:
vest_md_20 = gp.read_file("./raw-from-source/VEST/md_2020/md_2020.shp")

In [37]:
data_columns = [col for col in vest_md_20.columns if "G20" in col]

## Load, Clean and Process MD Day Of File

Note: Maryland splits election results between votes on the day of for the precincts and early, provisional, and absentee, which have to be downloaded separately.

In [65]:
#Load file
source_elections = pd.read_csv("./raw-from-source/Election_Results/Election_Day/All_By_Precinct_2020_General.csv")

#Clean the office name
source_elections["Office Name"] = source_elections["Office Name"].str.strip()

/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [66]:
#Define a function to swap position in a list
def swapPositions(list, pos1, pos2):
    list[pos1], list[pos2] = list[pos2], list[pos1]
    return list
 
# Swap the order of two counties (St. Mary's and Somerset - thrown off because of the "St.")
juris_list = swapPositions(sorted(vest_md_20["JURSCODE"].unique()), 18, 19)

In [41]:
display(source_elections)

,County,Election District,Election Precinct,Cong,Legs,Candidate Name,Party,Office Name,Office District,Winner,Write-In?,Early Voting Votes,Early Voting Votes Against,Election Day Votes,Election Day Votes Against,By Mail Votes,By Mail Votes Against,Prov. Votes,Prov. Votes Against,By Mail 2 Votes,By Mail 2 Votes Against,Total Votes,Total Votes Against
0,1,1,0,6,01C,Donald J. Trump,REP,President - Vice Pres,NaN,NaN,NaN,94,NaN,267,NaN,23,NaN,14,NaN,19,NaN,417.0,NaN
1,1,1,0,6,01C,Joe Biden,DEM,President - Vice Pres,NaN,Y,NaN,14,NaN,17,NaN,16,NaN,12,NaN,17,NaN,76.0,NaN
2,1,1,0,6,01C,Jo Jorgensen,LIB,President - Vice Pres,NaN,NaN,NaN,0,NaN,5,NaN,0,NaN,1,NaN,0,NaN,6.0,NaN
3,1,1,0,6,01C,Howie Gresham Hawkins,GRN,President - Vice Pres,NaN,NaN,NaN,0,NaN,2,NaN,0,NaN,0,NaN,0,NaN,2.0,NaN
4,1,1,0,6,01C,Jerome M. Segal,BAR,President - Vice Pres,NaN,NaN,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95278,24,7,1,1,38C,Mia Mason,DEM,Representative in Congress,01,NaN,NaN,475,NaN,188,NaN,313,NaN,83,NaN,473,NaN,1532.0,NaN
95279,24,7,1,1,38C,Other Write-Ins,BOT,Representative in Congress,01,NaN,Y,5,NaN,4,NaN,0,NaN,0,NaN,3,NaN,12.0,NaN
95280,24,7,1,1,38C,Brynja M. Booth,BOT,Judge Court of Appeals,01,Y,NaN,1365,283.0,609,124.0,321,85.0,142,26.0,688,109.0,3125.0,627.0
95281,24,7,1,1,38C,E. Gregory Wells,BOT,Judge Special Appeals At Large,NaN,Y,NaN,1395,246.0,606,120.0,339,65.0,132,32.0,687,107.0,3159.0,570.0


In [42]:
#Filter down to the relevant offices
source_elections = source_elections[source_elections["Office Name"].isin(['President - Vice Pres'])]

#Make a column that captures the county, election district, and precinct
source_elections["pivot_col"] = source_elections["County"].astype(str)+"-"+source_elections["Election District"].astype(str)+"-"+source_elections["Election Precinct"].astype(str)



In [ ]:
fips_dict = {'HOWA':"027", 'MONT':"031", 'PRIN':"033", 'STMA':"037",
             'QUEE':"035", 'WASH':"043", 'WICO':"045", 'CHAR':"017",
             'KENT':"029", 'CARO':"011", 'TALB':"041", 'DORC':"019", 
             'WORC':"047", 'SOME':"039", 'ALLE':"001", 'ANNE':"003",
             'BACI':"510", 'BACO':"005", 'CALV':"009", 'CARR':"013",
             'CECI':"015", 'FRED':"021", 'GARR':"023", 'HARF':"025"}

In [45]:
source_elections["Candidate Name"].unique()

array(['Donald J. Trump', 'Joe Biden', 'Jo Jorgensen',
       'Howie Gresham Hawkins', 'Jerome M. Segal', 'Sharon Wallace',
       'Dennis Andrew Ball', 'Barbara Bellar', 'President Boddie',
       'Mary Ruth Caro Simmons', 'Brian Carroll', 'Todd Cella',
       'Mark Charles', 'Phil Collins', 'Roque Rocky De La Fuente',
       'Ryan Ehrenreich', 'Randall Foltyniewkz', 'Tom Hoefling',
       'Shawn Howard', 'Johnson Lee', 'Susan B. Lochocki', 'Brock Pierce',
       'Deborah Rouse', 'Peter W. Sherrill', 'Jade Simmons',
       'Kasey Wells', 'Kanye West', 'Gloria La Riva ', 'Albert Raley',
       'Benjamin Schwalb', 'Edward Shlikas', 'Other Write-Ins'],
      dtype=object)

In [48]:
name_change_dict = {'Donald J. Trump':'G20PRERTRU', 
 'Joe Biden':'G20PREDBID', 
 'Jo Jorgensen':'G20PRELJOR',
'Howie Gresham Hawkins':'G20PREGHAW',
 'Jerome M. Segal':'G20PREBSEG'}

source_elections["Candidate Name"] = source_elections["Candidate Name"].map(name_change_dict).fillna('G20PREOWRI')

In [49]:
source_elections["Candidate Name"].unique()

array(['G20PRERTRU', 'G20PREDBID', 'G20PRELJOR', 'G20PREGHAW',
       'G20PREBSEG', 'G20PREOWRI'], dtype=object)

In [52]:
#Pivot the data
pivoted_results = pd.pivot_table(source_elections,index=["pivot_col"],columns=["Candidate Name"],values=['Total Votes'],aggfunc=sum)

#Clean up the columns and indices
pivoted_results = pivoted_results.fillna(0)
pivoted_results.columns = pivoted_results.columns.droplevel(0)
pivoted_results.reset_index(drop=False,inplace=True)

In [53]:
print(pivoted_results)

Candidate Name pivot_col  G20PREBSEG  G20PREDBID  G20PREGHAW  G20PRELJOR  \
0                  1-1-0         0.0        76.0         2.0         6.0   
1                 1-10-0         1.0       129.0         2.0         9.0   
2                 1-11-0         0.0       286.0         2.0         4.0   
3                 1-12-0         3.0       559.0        10.0        28.0   
4                 1-13-0         0.0       165.0         4.0         9.0   
...                  ...         ...         ...         ...         ...   
2030               9-8-3         1.0       962.0         5.0        11.0   
2031               9-8-4         4.0      3144.0        20.0        22.0   
2032               9-8-5         0.0       181.0         2.0         3.0   
2033               9-9-1         4.0      1229.0         5.0        15.0   
2034               9-9-2         3.0       873.0         8.0        19.0   

Candidate Name  G20PREOWRI  G20PRERTRU  
0                      0.0       417.0  
1    

In [54]:
def statewide_totals_check(partner_df,source_df,column_list):
    print("***Statewide Totals Check***")
    for race in column_list:
        if (partner_df[race].sum()- source_df[race].sum() != 0):
            print(race+" has a difference of "+str(partner_df[race].sum()-source_df[race].sum())+" votes")
            print("\tVEST: "+str(partner_df[race].sum())+" votes")
            print("\tSOURCES: "+str(source_df[race].sum())+" votes")
        else:
            print(race + " is equal", "\tVEST / RDH: " + str(partner_df[race].sum()))

In [55]:
statewide_totals_check(vest_md_20,pivoted_results,data_columns)

***Statewide Totals Check***
G20PREDBID is equal 	VEST / RDH: 1985023
G20PRERTRU has a difference of 2.0 votes
	VEST: 976414 votes
	SOURCES: 976412.0 votes
G20PRELJOR is equal 	VEST / RDH: 33488
G20PREGHAW is equal 	VEST / RDH: 15799
G20PREBSEG is equal 	VEST / RDH: 5884
G20PREOWRI has a difference of 380.0 votes
	VEST: 20423 votes
	SOURCES: 20043.0 votes


In [58]:
pivoted_results["County"] = pivoted_results["pivot_col"].apply(lambda x: x.split("-")[0])

In [60]:
pivoted_results["County"].unique()

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '2', '20', '21', '22', '23', '24', '3', '4', '5', '6', '7', '8',
       '9'], dtype=object)

In [56]:
def county_totals_check(partner_df,source_df,column_list,county_col,full_print=False):
    print("***Countywide Totals Check***")
    print("")
    diff_counties=[]
    for race in column_list:
        diff = partner_df.groupby([county_col]).sum()[race]-source_df.groupby([county_col]).sum()[race]
        for val in diff[diff != 0].index.values.tolist():
            if val not in diff_counties:
                diff_counties.append(val)
        if len(diff[diff != 0]!=0):   
            print(race + " contains differences in these counties:")
            for val in diff[diff != 0].index.values.tolist():
                county_differences = diff[diff != 0]
                print("\t"+val+" has a difference of "+str(county_differences[val])+" votes")
                print("\t\tVEST: "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
                print("\t\tSOURCES: "+str(source_df.groupby([county_col]).sum().loc[val,race])+" votes")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
        else:
            print(race + " is equal across all counties")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")

In [57]:
county_totals_check(partner_df,source_df,column_list,county_col,full_print=False)

NameError: name 'partner_df' is not defined